In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable, Any
import textstat
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
from daigt.features import fit_tokenizer
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
df = pd.read_parquet("input/white.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   row_id          221612 non-null  int32  
 1   essay_id        221612 non-null  object 
 2   generated       221612 non-null  int8   
 3   source          221612 non-null  object 
 4   prompt          221612 non-null  object 
 5   text            221612 non-null  object 
 6   text_bsc        221612 non-null  object 
 7   text_bow        221612 non-null  object 
 8   text_bow_len    221612 non-null  int16  
 9   prompt_bsc      221612 non-null  object 
 10  prompt_bow      221612 non-null  object 
 11  prompt_bow_len  221612 non-null  int16  
 12  white_sim       221612 non-null  float32
dtypes: float32(1), int16(2), int32(1), int8(1), object(8)
memory usage: 16.3+ MB


# Character level features

In [3]:
%%time
text_col = "text_bsc"
col = "ch_len"
df[col] = df[text_col].str.len()
df[col] = df[col].astype(np.int32)

def digit_frac(row) -> float:
    return pen.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return pen.letter_frac(row[text_col])


def space_frac(row) -> float:
    return pen.space_frac(row[text_col])


def punc_frac(row) -> float:
    return pen.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return pen.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return pen.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return pen.repeat_substring_frac(row[text_col])


char_fns: Dict[str, Callable] = {
    "ch_digit_frac": digit_frac,
    "ch_letter_frac": letter_frac,
    "ch_space_frac": space_frac,
    "ch_punc_frac": punc_frac,
    "ch_upper_frac": upper_frac,
    "ch_repeat_char_frac": repeat_char_frac,
    #"ch_repeat_substring_frac": repeat_substring_frac,
}

for col, fn in char_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ch_digit_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:25<00:00, 8727.65it/s]


ch_letter_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:28<00:00, 7722.66it/s]


ch_space_frac


...

ch_upper_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:25<00:00, 8532.49it/s]


ch_repeat_char_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:20<00:00, 2743.91it/s]

CPU times: user 3min 32s, sys: 991 ms, total: 3min 33s
Wall time: 3min 34s


# Word and Sentence Features

In [4]:
text_col = "text_bsc"

def sent_len_mean(row) -> float:
    return pen.sentence_length_mean(row[text_col])


def sent_len_std(row) -> float:
    return pen.sentence_length_std(row[text_col])


def sent_len_delta_mean(row) -> float:
    return pen.sentence_length_delta_mean(row[text_col])


def sent_len_delta_std(row) -> float:
    return pen.sentence_length_delta_std(row[text_col])


fmap: Dict[str, Callable] = {
    "ws_sent_len_mean": sent_len_mean,
    "ws_sent_len_std": sent_len_std,
    "ws_sent_len_delta_mean": sent_len_delta_mean,
    "ws_sent_len_delta_std": sent_len_delta_std,
}
for col, fn in fmap.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ws_sent_len_mean


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:48<00:00, 4530.26it/s]


ws_sent_len_std


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:52<00:00, 4221.23it/s]


ws_sent_len_delta_mean


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:49<00:00, 4450.18it/s]


ws_sent_len_delta_std


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:52<00:00, 4226.02it/s]


# Textstat Features

In [5]:
text_col = "text_bsc"

def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)
    

def polysyllable_frac(row) -> float:
    return textstat.polysyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def monosyllable_frac(row) -> float:
    return textstat.monosyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def difficult_words(row) -> float:
    return textstat.difficult_words(row[text_col])


def spache_readability(row) -> float:
    return textstat.spache_readability(row[text_col])


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


stage1: List[Tuple[str, Callable, Any]] = [
    ("ts_syllable_count", syllable_count, np.int32),
    ("ts_lexicon_count", lexicon_count, np.int32),
    ("ts_sentence_count", sentence_count, np.int32),
]
stage2: List[Tuple[str, Callable, Any]] = [
    ("ts_syllables_per_word", syllables_per_word, np.float32),
    ("ts_syllables_per_sent", syllables_per_sent, np.float32),
    ("ts_words_per_sent", words_per_sent, np.float32),
    ("ts_polysyllable_frac", polysyllable_frac, np.float32),
    ("ts_monosyllable_frac", monosyllable_frac, np.float32),
    ("ts_flesch_reading_ease", flesch_reading_ease, np.float32),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade, np.float32),
    ("ts_gunning_fog", gunning_fog, np.float32),
    ("ts_smog_index", smog_index, np.float32),
    ("ts_automated_readability_index", automated_readability_index, np.float32),
    ("ts_coleman_liau_index", coleman_liau_index, np.float32),
    ("ts_linsear_write_formula", linsear_write_formula, np.float32),
    ("ts_dale_chall_readability_score", dale_chall_readability_score, np.float32),
    ("ts_difficult_words", difficult_words, np.float32),
    ("ts_spache_readability", spache_readability, np.float32),
    ("ts_mcalpine_eflaw", mcalpine_eflaw, np.float32),
]
for col, fn, dtype in stage1:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)
for col, fn, dtype in stage2:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)

ts_syllable_count


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:37<00:00, 2278.43it/s]


ts_lexicon_count


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:16<00:00, 13280.79it/s]


ts_sentence_count


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:41<00:00, 2189.96it/s]


ts_spache_readability


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [02:34<00:00, 1434.42it/s]


ts_mcalpine_eflaw


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:01<00:00, 3627.40it/s]


# VAD Features

In [6]:
vdf = pd.read_csv("input/NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt", header=0, names=["term", "valence", "arousal", "dominance"], 
                 sep="\t", engine="c", low_memory=False)
cols = ["valence", "arousal", "dominance"]
vdf[cols] = vdf[cols].astype(np.float32)
vdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19970 entries, 0 to 19969
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   term       19969 non-null  object 
 1   valence    19970 non-null  float32
 2   arousal    19970 non-null  float32
 3   dominance  19970 non-null  float32
dtypes: float32(3), object(1)
memory usage: 390.2+ KB


In [7]:
vad: Dict[str, Tuple] = {}
for t in tqdm(vdf.itertuples()):
    v = getattr(t, "valence")
    a = getattr(t, "arousal")
    d = getattr(t, "dominance")
    t = str(getattr(t, "term")).strip().lower()
    vad[t] = (v, a, d)

19970it [00:00, 1557837.54it/s]


In [8]:
cols = ["va_valence_mean", "va_valence_std", "va_arousal_mean", "va_arousal_std", "va_dominance_mean", "va_dominance_std"]
rows = []
for text in tqdm(df["text_bow"]):
    vas, ars, dos = [], [], []
    tokens = text.split()
    for t in tokens:
        if t in vad:
            v, a, d = vad[t]
            vas.append(v)
            ars.append(a)
            dos.append(d)
    if len(vas) == 0:
        vas = [-1]
    if len(ars) == 0:
        ars = [-1]
    if len(dos)==0:
        dos = [-1]
    rows.append([np.mean(vas), np.std(vas), np.mean(ars), np.std(ars), np.mean(dos), np.std(dos)])
df[cols] = rows
df[cols] = df[cols].astype(np.float32)
del rows, vdf, vad
gc.collect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:41<00:00, 5390.31it/s]


32

# TF-IDF Features

In [9]:
%%time
corpus = df["text_bow"].tolist()
tokenizer = fit_tokenizer(corpus=corpus)
dp = "output/tokenizer"
Path(dp).mkdir(parents=True, exist_ok=True)
tokenizer.save_pretrained(dp)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")




PreTrainedTokenizerFast(name_or_path='', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']
CPU times: user 13min 30s, sys: 2.54 s, total: 

In [10]:
%%time
tokenized = [tokenizer.tokenize(x) for x in corpus]
print(tokenized[0])

['Ġcars', 'Ġcars', 'Ġhave', 'Ġbeen', 'Ġaround', 'Ġsince', 'Ġthey', 'Ġbecame', 'Ġfamous', 'Ġin', 'Ġthe', 'Ġ1900', 's', 'Ġwhen', 'Ġhenry', 'Ġford', 'Ġcreated', 'Ġand', 'Ġbuilt', 'Ġthe', 'Ġfirst', 'Ġmodel', 't', 'Ġcars', 'Ġhave', 'Ġplayed', 'Ġa', 'Ġmajor', 'Ġrole', 'Ġin', 'Ġour', 'Ġevery', 'Ġday', 'Ġlives', 'Ġsince', 'Ġthen', 'Ġbut', 'Ġnow', 'Ġpeople', 'Ġare', 'Ġstarting', 'Ġto', 'Ġquestion', 'Ġif', 'Ġlimiting', 'Ġcar', 'Ġusage', 'Ġwould', 'Ġbe', 'Ġa', 'Ġgood', 'Ġthing', 'Ġto', 'Ġme', 'Ġlimiting', 'Ġthe', 'Ġuse', 'Ġof', 'Ġcars', 'Ġmight', 'Ġbe', 'Ġa', 'Ġgood', 'Ġthing', 'Ġto', 'Ġdo', 'Ġin', 'Ġlike', 'Ġmatter', 'Ġof', 'Ġthis', 'Ġarticle', 'Ġin', 'Ġgerman', 'Ġsuburb', 'Ġlife', 'Ġgoes', 'Ġon', 'Ġwithout', 'Ġcars', 'Ġby', 'Ġelizabeth', 'Ġrosenthal', 'Ġstates', 'Ġhow', 'Ġautomobiles', 'Ġare', 'Ġthe', 'Ġlin', 'ch', 'pin', 'Ġof', 'Ġsuburbs', 'Ġwhere', 'Ġmiddle', 'Ġclass', 'Ġfamilies', 'Ġfrom', 'Ġeither', 'Ġshanghai', 'Ġor', 'Ġchicago', 'Ġtend', 'Ġto', 'Ġmake', 'Ġtheir', 'Ġhomes', 'Ġexperts', 'Ġs

In [11]:
%%time
vectorizer = TfidfVectorizer(
    ngram_range=(1,1), 
    lowercase=False, 
    sublinear_tf=True, 
    analyzer = 'word',
    tokenizer = lambda x: x,
    preprocessor = lambda x: x,
    token_pattern =None, 
    strip_accents=None
)
x = vectorizer.fit_transform(tokenized)
print(f"x={x.shape}")

x=(221612, 30457)
CPU times: user 24.4 s, sys: 548 ms, total: 25 s
Wall time: 25 s


In [12]:
%%time
cols = [f"tf_{f}" for f in vectorizer.get_feature_names_out()]
df[cols] = scipy.sparse.csr_matrix(x).todense().astype(np.float32)
df = df.copy()  # defragment

CPU times: user 1min 19s, sys: 30.6 s, total: 1min 49s
Wall time: 2min 6s


In [13]:
vocab = vectorizer.vocabulary_
print(f"{len(vocab):,} vocab")
idf = vectorizer.idf_
print(f"idf={idf.shape}")
with open("output/tfidf.json", "w") as f:
    json.dump({"vocabulary": vocab, "idf": idf.tolist()}, f)

30,457 vocab
idf=(30457,)


In [14]:
del vectorizer, corpus, vocab, idf
gc.collect()

32

# Review Data

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 30506 entries, row_id to tf_Ġzyg
dtypes: float32(30490), int16(2), int32(5), int8(1), object(8)
memory usage: 25.2+ GB


In [16]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_Ġhe"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
df[features].describe(percentiles=percentiles)

,ch_digit_frac,ch_len,ch_letter_frac,ch_punc_frac,ch_repeat_char_frac,ch_space_frac,ch_upper_frac,tf_Ġhe,tf_Ġhead,tf_Ġheadache,tf_Ġheadaches,tf_Ġheaded,tf_Ġheading,tf_Ġheadlines,tf_Ġheadphones,tf_Ġheadquart,tf_Ġheadquartered,tf_Ġheadquarters,tf_Ġheads,tf_Ġheal,tf_Ġhealed,tf_Ġhealing,tf_Ġhealth,tf_Ġhealthcare,tf_Ġhealthier,tf_Ġhealthy,tf_Ġhealy,tf_Ġhear,tf_Ġheard,tf_Ġhearing,tf_Ġhearings,tf_Ġhears,tf_Ġheart,tf_Ġheartbeat,tf_Ġhearted,tf_Ġhearts,tf_Ġheat,tf_Ġheated,tf_Ġheath,tf_Ġheather,tf_Ġheating,tf_Ġheav,tf_Ġheaven,tf_Ġheavenly,tf_Ġheavens,tf_Ġheavier,tf_Ġheavily,tf_Ġheavy,tf_Ġheb,tf_Ġhebrew,tf_Ġhebrews,tf_Ġheck,tf_Ġhect,tf_Ġhectares,tf_Ġhectic,tf_Ġhector,tf_Ġhed,tf_Ġhedge,tf_Ġhedging,tf_Ġheed,tf_Ġheels,tf_Ġhefty,tf_Ġhege,tf_Ġhegel,tf_Ġhegemon,tf_Ġhegemonic,tf_Ġhegemony,tf_Ġheid,tf_Ġheide,tf_Ġheidegger,tf_Ġheidel,tf_Ġheidelberg,tf_Ġheidrun,tf_Ġheight,tf_Ġheighten,tf_Ġheightened,tf_Ġheights,tf_Ġheil,tf_Ġhein,tf_Ġheine,tf_Ġheineken,tf_Ġheinem,tf_Ġheinemann,tf_Ġheinous,tf_Ġheinz,tf_Ġheir,tf_Ġhel,tf_Ġheld,tf_Ġhelen,tf_Ġhelic,tf_Ġhelicop,tf_Ġhelicopter,tf_Ġhelicopters,tf_Ġhelium,tf_Ġhell,tf_Ġhellen,tf_Ġhellenistic,tf_Ġheller,tf_Ġhello,tf_Ġhelm,tf_Ġhelmet,tf_Ġhelp,tf_Ġhelped,tf_Ġhelper,tf_Ġhelpful,tf_Ġhelping,tf_Ġhelpless,tf_Ġhelplessness,tf_Ġhelps,tf_Ġhem,tf_Ġhemat,tf_Ġheming,tf_Ġhemingway,tf_Ġhemis,tf_Ġhemisphere,tf_Ġhemod,tf_Ġhemodialysis,tf_Ġhemoglobin,tf_Ġhemorrh,tf_Ġhemorrhage,tf_Ġhemp,tf_Ġhen,tf_Ġhence,tf_Ġhenceforth,tf_Ġhend,tf_Ġhenderson,tf_Ġhendric,tf_Ġhenn,tf_Ġhenri,tf_Ġhenry,tf_Ġhens,tf_Ġhep,tf_Ġhepat,tf_Ġhepatitis,tf_Ġher,tf_Ġherald,tf_Ġherb,tf_Ġherbal,tf_Ġherbert,tf_Ġherbs,tf_Ġherd,tf_Ġhere,tf_Ġhered,tf_Ġhereditary,tf_Ġheredity,tf_Ġherein,tf_Ġherit,tf_Ġheritage,tf_Ġherm,tf_Ġherman,tf_Ġhermene,tf_Ġhern,tf_Ġhernand,tf_Ġhernandez,tf_Ġhero,tf_Ġherod,tf_Ġheroes,tf_Ġheroic,tf_Ġheroin,tf_Ġheroine,tf_Ġheroism,tf_Ġhers,tf_Ġherself,tf_Ġhershey,tf_Ġherz,tf_Ġherzberg,tf_Ġhes,tf_Ġhesit,tf_Ġhesitant,tf_Ġhesitate,tf_Ġhesitation,tf_Ġhess,tf_Ġhester,tf_Ġheter,tf_Ġheterogeneity,tf_Ġheterogeneous,tf_Ġheterosexual,tf_Ġheur,tf_Ġheuristic,tf_Ġheuristics,tf_Ġhew,tf_Ġhewitt,tf_Ġhewlett,tf_Ġhex,tf_Ġhey,tf_Ġhezb,tf_Ġhezbollah,ts_automated_readability_index,ts_coleman_liau_index,ts_dale_chall_readability_score,ts_difficult_words,ts_flesch_kincaid_grade,ts_flesch_reading_ease,ts_gunning_fog,ts_lexicon_count,ts_linsear_write_formula,ts_mcalpine_eflaw,ts_monosyllable_frac,ts_polysyllable_frac,ts_sentence_count,ts_smog_index,ts_spache_readability,ts_syllable_count,ts_syllables_per_sent,ts_syllables_per_word,ts_words_per_sent,va_arousal_mean,va_arousal_std,va_dominance_mean,va_dominance_std,va_valence_mean,va_valence_std,ws_sent_len_delta_mean,ws_sent_len_delta_std,ws_sent_len_mean,ws_sent_len_std
count,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.000000,221612.0000

In [17]:
%%time
df.to_parquet(f"output/features.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 1min 7s, sys: 6.32 s, total: 1min 13s
Wall time: 1min 16s


In [18]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:56:59.878683
